# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [32]:
!pip install xlsxwriter
!pip install python-dotenv

import os
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

from dotenv import load_dotenv
load_dotenv()

True

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [33]:
pwd = '/content/trading_practice/algorithmic-trading-python/starter_files'

stocks = pd.read_csv(f'{pwd}/sp_500_stocks.csv')
print(len(stocks.index))

token = os.getenv("IEX_CLOUD_API_TOKEN")
print(token)

505
Tpk_059b97af715d417d9f49f50b51b1c448


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [34]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={token}'
data = requests.get(api_url).json()
data

{'avg10Volume': 119834751,
 'avg30Volume': 105976295,
 'beta': 1.1604944469697382,
 'companyName': 'Apple Inc',
 'day200MovingAvg': 111.44,
 'day30ChangePercent': 0.1459776907584099,
 'day50MovingAvg': 124.67,
 'day5ChangePercent': 0.02576327575950595,
 'dividendYield': 0.006170615466735877,
 'employees': 0,
 'exDividendDate': '2020-10-30',
 'float': 0,
 'marketcap': 2276331903421,
 'maxChangePercent': 51.1884582916017,
 'month1ChangePercent': 0.14845531056050804,
 'month3ChangePercent': 0.2224686981185922,
 'month6ChangePercent': 0.4803380908114314,
 'nextDividendDate': '0',
 'nextEarningsDate': '0',
 'peRatio': 40.17798567622653,
 'sharesOutstanding': 17251244576,
 'ttmDividendRate': 0.8436780471328394,
 'ttmEPS': 3.4,
 'week52change': 0.8598206070657568,
 'week52high': 136.46,
 'week52low': 56.59,
 'year1ChangePercent': 0.9025348615486172,
 'year2ChangePercent': 2.695911812730355,
 'year5ChangePercent': 4.467055699785349,
 'ytdChangePercent': 0.8530832572544891}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [35]:
data['year1ChangePercent']

0.9025348615486172

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [36]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [47]:
import json

df = pd.DataFrame(columns = my_columns)

batch_url = 'https://sandbox.iexapis.com/stable/stock/market/batch'

for symbol_string in symbol_strings:
    batch_api_call_url = f'{batch_url}/?types=stats,quote&symbols={symbol_string}&token={token}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        df = df.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ], 
                index = my_columns), 
                ignore_index = True
                )
        
    
df

,Ticker,Price,One-Year Price Return,Shares to Buy
0,A,120.15,0.401226,N/A
1,AAL,15.66,-0.461716,N/A
2,AAP,166.22,0.00872566,N/A
3,AAPL,134.99,0.878473,N/A
4,ABBV,107.24,0.21744,N/A
...,...,...,...,...
500,YUM,108.15,0.0941846,N/A
501,ZBH,156.38,-0.00127377,N/A
502,ZBRA,378.26,0.482336,N/A
503,ZION,44.12,-0.128201,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [51]:
df.sort_values('One-Year Price Return', ascending = False, inplace = True)
df = df[:50]

df = df.reset_index(level=0, drop=True)

df.head()

,Ticker,Price,One-Year Price Return,Shares to Buy
0,CARR,38.30,2.2483,N/A
1,PYPL,249.63,1.22704,N/A
2,NVDA,538.80,1.21789,N/A
3,LB,40.16,1.17284,N/A
4,ALB,156.35,1.114,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [39]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

NameError: name 'writer' is not defined

## Saving Our Excel Output

As before, saving our Excel output is very easy: